# 主机名和域名区别

## 主机名

在局域网中，每台主机都有一个主机名，可以使用英文字母或者单词组成的主机名来代替该主机的IP地址.

实际上通过`ping 主机名`和`ping IP`效果是一样的. 

在实际开发过程中，通常使用`gethostbyname`和`gethostbyaddr`这两个系统函数，其中:
* gethostbyname: 表示通过主机名来查询IP地址，即传入一个主机名或域名，它就给你返回一个IP址；
* gethostbyaddr: 表示通过IP地址来查询主机名；
在ping命令的源码中可以看到它就调用了`gethostbyname`函数将主机名转换成IP地址，然后才进行的ping操作。

如果局域网内的所有主机都想使用域名来代替IP进行通信，那么要求该局域网中每个主机上的hosts文件都要把本局域网中的所有主机的主机名和IP的对应关系都存起来。当局域网内的主机非常多的时候，维护起来就非常麻烦，例如IP有变更的时候就需要挨个把所有的主机的hosts文件都更新一遍。

那么有没有更好的办法来解决这个问题呢？

方法就是通过搭建内网DNS服务来完成主机名和IP的映射；

## 域名

域名是Internet网络上的一个服务器或一个网络系统的名字，即在因特网范围内某一个服务器的名字.

域名是由一串用点分隔的名字组成的，通常包含组织名，而且始终包括两到三个字母的后缀，以指明组织的类型或该域所在的国家或地区。

域名可以认为是主机在公网环境中的标识，在在公网下，对应一个唯一的IP,在全世界，没有重复的域名。

域名的形式是以若干个英文字母和数字组成，由“.”分隔成几部分，如IBM.COM就是一个域名。

.中国和.com的管理机构是不同的
* 中文后缀的是由CNNIC管理的
* 英文后缀的是由NSI管理的。

域名的范围要比主机名大。

一个域名下可以有多个主机名，域名下还可以有子域名。例如，域名abc.com下，有主机server1和server2，其主机全名就是server1.abc.com和server2.abc.com。

每一级域名长度的限制是63个字符，域名总长度则不能超过253个字符。

注意:
1. IP地址为通讯的根本。
2. IP地址并不代表一台具体的主机，而是代表一台主机与一特定网络的链接。

所以：
1. 一台主机可拥有多个IP（防止一些网络不能用），所以一个域名可以有多个IP，但在一个时刻只能用一个IP。
2. 一个IP只能作用一个站点，但一个站点内可以有多台计算机，此为网络地址转换（HAT）技术，无线WiFi技术的原理。

## 主机名和域名区别

共性：都能对应到一个唯一的IP上

区别: 主机名用于局域网中, 域名用于公网中

In [14]:
import socket
import getpass
import platform
import uuid


host_name = socket.gethostname()
print(host_name)
print(socket.getfqdn(host_name))

ip  = socket.gethostbyname(host_name)
print(ip)

ip_list = socket.gethostbyname_ex(host_name)
print(ip_list)

user=getpass.getuser()
print(user)

my_architecture=platform.architecture()
print(my_architecture)

my_platform=platform.platform()
print(my_platform)


mac=uuid.UUID(int=uuid.getnode())
MAC=mac.hex[-12:]
print(mac)
print(MAC)

import os

print(os.name)

xxx.xx-PC
xxx.xx-PC
192.168.96.1
('xxx.xx-PC', [], ['192.168.48.1', '192.168.11.1', '192.168.96.1', '192.168.1.115'])
xxx.xx
('64bit', 'WindowsPE')
Windows-10-10.0.17134-SP0
00000000-0000-0000-0000-509a4c220938
509a4c220938
nt


## hosts文件

windows和linux系统中都有个hosts文件来记录本机所在局域网中的主机的IP地址和对应的主机名

host文件由一行行的IP和主机名的对应组成，形式如: 127.0.0.1 localhost


###  优雅获取本机 IP 方法

* 不推荐：靠猜测去获取本地IP方法
    如果机器上只有eth0 或者 只有bond0上有IP，那么此类代码都有可能失败，而且还不容易移植到其他平台上。
* 不推荐：通过hostname来获取本机IP
    通过获取hostname，然后再用hostname反查机器的IP。不推荐, 因为很多的机器没有规范这个hostname的设置。
* 推荐: 通过 UDP 获取本机 IP，目前见过最优雅的方法    
    * 没有任何的依赖，也没有去猜测机器上的网络设备信息。
    * 利用 UDP 协议来实现的，生成一个UDP包，把自己的 IP 放如到 UDP 协议头中，然后从UDP包中获取本机的IP。
    * 这个方法并不会真实的向外部发包，所以抓包工具是看不到的。但会申请一个 UDP 的端口，如果经常调用会比较耗时的，可以将查询到的IP缓存起来，性能可以获得很大提升。

In [13]:
import socket

# 通过 UDP 获取本机 IP，目前见过最优雅的方法 
def get_host_ip():
    try:
        s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
        s.connect(('8.8.8.8', 80))
        ip = s.getsockname()[0]
    finally:
        s.close()
    return ip

print(get_host_ip())

192.168.1.115


## DNS

DNS（Domain Name Server，域名服务器）是进行域名(domain name)和与之相对应的IP地址 (IP address)转换的服务器。DNS中保存了一张域名(domain name)和与之相对应的IP地址 (IP address)的表，以解析消息的域名。

在Internet上域名与IP地址之间是一对一（或者多对一）的，域名虽然便于人们记忆，但机器之间只能互相认识IP地址，它们之间的转换工作称为域名解析，域名解析需要由专门的域名解析服务器来完成，DNS就是进行域名解析的服务器。域名的最终指向是IP。

DNS使用TCP和UDP端口53

DNS主要基于UDP运输层协议, 只会在UDP报文中表明有截断的时候使用TCP查询。

一次UDP名字服务器交换可以短到两个包：一个查询包、一个响应包。一次TCP交换则至少包含9个包：三次握手初始化TCP会话、一个查询包、一个响应包以及四次分手的包交换。

考虑到效率原因，TCP连接的开销大得，故采用UDP作为DNS的运输层协议，这也将导致只有13个根域名服务器的结果。

### 什么是TTL？

TTL值全称是“生存时间（Time To Live)”，简单的说它表示DNS记录在DNS服务器上缓存时间。

这个缓存时间太长和太短都不好:
* 缓存时间太长，一旦域名被解析到的IP有变化，会导致被客户端缓存的域名无法解析到变化后的IP地址，以致该域名不能正常解析，这段时间内有可能会有一部分用户无法访问网站。
* 缓存时间太短，会导致用户每次访问网站都要重新解析一次域名。


### WHOIS（域名数据库查询）

一个域名的所有者可以通过查询WHOIS数据库而被找到；

对于大多数根域名服务器，基本的WHOIS由ICANN维护，而WHOIS的细节则由控制那个域的域注册机构维护。

对于240多个国家代码顶级域名（ccTLDs），通常由该域名权威注册机构负责维护WHOIS。例如:
* 中国互联网络信息中心（China Internet Network Information Center）负责.CN域名的WHOIS维护，
* 香港互联网注册管理有限公司（Hong Kong Internet Registration Corporation Limited）负责.HK域名的WHOIS维护，
* 台湾网络信息中心（Taiwan Network Information Center）负责.TW域名的WHOIS维护。